In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)

# Introduction

* hook : a h
* research problem :
* research question :
* thesis :
* overview : 

The broad question that I will address is "Question 1: What player characteristics and behaviors are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?". To make it more specific, I made the question "Can the player’s age and total hours played predict their likelihood of subscribing to the game-related newsletter?". 

The subscribe column will still be the target variable (whether a player subscribed to the newsletter), but we are focusing on the predictors of experience and age as explanatory variables. The age variable will be treated as a continuous feature, and experience will be analyzed categorically. Since the goal is to predict subscription likelihood based on these two factors, I would use K-Nearest Neighbors for this task. I'd use the KNN method since it can handle non-linear patterns as well as the experience categories (non-numeric), showing how age and experience influence the likelihood of subscription.

# Data Description

First, upon seeing the 2 available datasets, there is a `players.csv` file that includes data about individual players, such as name, experience, playtime, subscription status, gender, age, and hashed email. The total amount of observations is 197, and number of variables is 7. The `age` variable is numerical, and we can calculate the mean and standard deviation to better understand player demographics. The `subscription_status` variable is a boolean (true or false), which could be important to our analysis of predicting newsletter subscriptions. The player experience variable is categorical, representing player classifications which we need. Some potential issues with this data include missing values in certain columns, such as the age category in `players.csv`. There are 2 observations with `N/A` in that column.

The `sessions.csv` file records individual play sessions, with start and end times variables. There is a hashed email variable, start time and end time, as well as the original start and end time. There are five variables total. The dataset mainly contains observations on the time players put into the game. The start and end times are in a date format, which is not numerical, while the original start and end times are numerical which measures how long a player was engaged in a session  which reflects the demand on the server during gameplay. However, these values are extremely big and very hard to read, so we would need to standardize the data.

# Exploratory Analysis and Visualization 

**Reading Data**

To read in the data, we first loaded in the `tidyverse` library. `tidyverse` is a package in the R coding language used in data science for reading, wrangling, analysis, and visualization. 

Data was collected from two files : `players.csv`, which informs us of participant characteristics, and `sessions.csv`, which details every individual play session. In order to maintain reproducability, we published these files into our Github repository and read them in using a URL. The resulting datasets were named `players` and `sessions`, respectively.

The data was then made accessible for `tidyverse` functions ('tidied') before it was displayed. In our case, this meant reformatting the `hashedEmail` column to have underscores (`_`) between its words and using `as.POSIXct()` to reformat the start and end times in `sessions` to be `dttm` variables, which represent dates.

Finally, `players` and `sessions` were displayed using the `head()` function, which shows the first six rows of a dataset.

In [ ]:
library(tidyverse)

players_url <- "https://raw.githubusercontent.com/20under20/dcsi100-project-group-002-13/refs/heads/main/players.csv"


players <- read_csv(players_url, show_col_types = FALSE) |>        # reading the 'players' dataset
           rename(hashed_email = hashedEmail,                      # tidying the 'players' dataset
                  age = Age) 
head(players)                                                      # displaying the 'players' dataset

sessions_url <- "https://raw.githubusercontent.com/20under20/dcsi100-project-group-002-13/refs/heads/main/sessions.csv"

sessions <- read_csv(sessions_url, show_col_types = FALSE) |>       # reading the 'sessions' dataset
            rename(hashed_email = hashedEmail) |>                   # tidying the 'sessions' dataset 
            mutate(start_time = as.POSIXct(start_time),
                   end_time = as.POSIXct(end_time)) 
head(sessions)                                                      # displaying the 'sessions' dataset

In [ ]:
data <- read_csv("https://raw.githubusercontent.com/brittany-tsui/csv-files/refs/heads/main/players.csv")

data_mean <- data |>
  select(played_hours, Age) |>
  summarize(played_hours = mean(played_hours, na.rm = TRUE),
    Age = mean(Age, na.rm = TRUE))
data_mean

data_bar <- ggplot(data, aes(x = Age, y = played_hours)) + 
geom_bar(stat = "identity") +
xlab("Age (Years)") +
ylab("Played Hours (Hours)") + 
ggtitle("Age vs. Played Hours")

data_bar

From these plots we can see that the gamers who are 17 played the most hours, hinting that the chance of subscribing to the newsletter could be related to this age and the hours played. However, we would need to factor in whether they are subscribed or not, and separate the sections into telling whether they are or not. This visualization just demonstrates the age and the hours played.

Methods (4)

I would suggest using the K-Nearest Neighbors (KNN) algorithm. 
- KNN is suitable here because it doesn't need a linear representation for the relationship between predictors (played hours and age) and outcome (subscription), thus apt to identify potential non-linear relationships.
- KNN selects the 'k' closest data points within the feature space and classifies based upon the majority class of the neighbors. The reason for this flexibility is that the interaction between age, played hours, and subscription may not be linear.
- The method uses the premise that the distance between points can be used to make predictions and that features must be scaled to have a meaningful distance metric.
One drawback of KNN is its computational cost, especially for large datasets because it must calculate distances from test data to all training points.

To compare and select the best model, I will compare its performance using cross-validation. For pre-processing the data, do na.rm = TRUE for the missing values, standardize the data for age and playing hours, and then see if there is any variance in people's experiences if they are subscribed. The information will be split into a training set (70%) and a testing set (30%), and I'll also use k-fold cross-validation to achieve a better estimate. This should allow us to identify predictors that make the most contributions to subscription probability!